# 1. Importation des librairies

In [183]:
import pandas as pd
import numpy as np

# 2. Importation du dataset

In [184]:
data_set_path = '~/project9/V1/dataset/'
population_df = pd.read_csv(data_set_path+'population.csv')
dispo_alim_df  = pd.read_csv(data_set_path+'dispo-alim.csv')
eco_df = pd.read_csv(data_set_path + 'données-eco.csv')

# 3. Filtre des années et du type de produit et Suppression des colonnes inutiles

In [185]:
population_df = pd.DataFrame({'Pays' : population_df.Zone, 'Année' : population_df['Année'],'Population' : population_df.Valeur *1000})
population_df = population_df[population_df['Année']== 2017]
dispo_alim_df = dispo_alim_df[dispo_alim_df['Produit'].str.contains('Volailles',na= False)]
dispo_alim_df = pd.DataFrame({'Pays' : dispo_alim_df.Zone, 'Élément' : dispo_alim_df['Élément'],'Produit' : dispo_alim_df['Produit'], 'Année' : dispo_alim_df['Année'], 'Valeur' : dispo_alim_df['Valeur']})

display(dispo_alim_df['Élément'].drop_duplicates().to_list())

['Production',
 'Importations - Quantité',
 'Variation de stock',
 'Disponibilité intérieure',
 'Pertes',
 'Résidus',
 'Nourriture',
 'Disponibilité alimentaire en quantité (kg/personne/an)',
 'Disponibilité alimentaire (Kcal/personne/jour)',
 'Disponibilité de protéines en quantité (g/personne/jour)',
 'Disponibilité de matière grasse en quantité (g/personne/jour)',
 'Exportations - Quantité',
 'Alimentation pour touristes',
 'Traitement',
 'Autres utilisations (non alimentaire)',
 'Aliments pour animaux',
 'Semences']

# 4. Transposition du tableau disponibilité alimentaire

In [186]:
usefull_element = ['Importations - Quantité','Production','Disponibilité intérieure','Nourriture']


final_df = dispo_alim_df[dispo_alim_df['Élément'] == 'Disponibilité de protéines en quantité (g/personne/jour)']
final_df = final_df.drop(columns=['Élément'])
final_df = final_df.rename(columns={'Valeur' : 'Disponibilité de protéines en quantité (g/personne/jour)'})

display(final_df)
def partition(element_tab,dispo_df,final_df):
    for x in element_tab:
        inter_df = dispo_df[dispo_df['Élément'] == x]
        inter_df = inter_df.drop(columns = ['Élément'])
        inter_df = inter_df.rename(columns = {'Valeur' : x})
        final_df = pd.merge(final_df,inter_df,how='outer',on=['Pays','Produit','Année'])
    return final_df

final_df = partition(usefull_element,dispo_alim_df,final_df)


usefull_element

,Pays,Produit,Année,Disponibilité de protéines en quantité (g/personne/jour)
660,Afghanistan,Viande de Volailles,2017,0.54
1715,Afrique du Sud,Viande de Volailles,2017,14.11
2722,Albanie,Viande de Volailles,2017,6.26
3783,Algérie,Viande de Volailles,2017,1.97
4883,Allemagne,Viande de Volailles,2017,7.96
...,...,...,...,...
172339,Venezuela (République bolivarienne du),Viande de Volailles,2017,7.23
173398,Viet Nam,Viande de Volailles,2017,4.01
174327,Yémen,Viande de Volailles,2017,3.04
175372,Zambie,Viande de Volailles,2017,1.15


['Importations - Quantité',
 'Production',
 'Disponibilité intérieure',
 'Nourriture']

# 5. Fusion population et table disponibilité alimentaire final + mise en avant des pays sans data en disponibilité alimentaire

In [187]:
final_df = pd.merge(final_df,population_df,how='outer',on=['Pays','Année'])


no_data_alim_df = final_df[final_df['Produit'].isna()]
final_df = final_df[final_df['Produit'].notna()]



# 6. Intégration du PIB

In [188]:
display(eco_df.columns)

eco_df = pd.DataFrame({'Pays' : eco_df.Zone, 'Produit intérieur brut en dollars' : eco_df.Valeur})

final_df = pd.merge(final_df,eco_df,how='outer',on=['Pays'])


Index(['Code Domaine', 'Domaine', 'Code zone (M49)', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Note'],
      dtype='object')

# 7. Quick tour des données

In [189]:
final_df.head()

,Pays,Produit,Année,Disponibilité de protéines en quantité (g/personne/jour),Importations - Quantité,Production,Disponibilité intérieure,Nourriture,Population,Produit intérieur brut en dollars
0,Afghanistan,Viande de Volailles,2017.0,0.54,29.0,28.0,57.0,55.0,36296113.0,1.889635e+04
1,Afrique du Sud,Viande de Volailles,2017.0,14.11,514.0,1667.0,2118.0,2035.0,57009756.0,3.490067e+05
2,Albanie,Viande de Volailles,2017.0,6.26,38.0,13.0,47.0,47.0,2884169.0,1.301973e+04
3,Algérie,Viande de Volailles,2017.0,1.97,2.0,275.0,277.0,264.0,41389189.0,1.700970e+05
4,Allemagne,Viande de Volailles,2017.0,7.96,842.0,1514.0,1739.0,1609.0,82658409.0,3.690849e+06


Quelles sont les dimensions de la table ?

In [190]:
final_df.shape

(212, 10)

Indications générales sur la table : 

In [191]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 10 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Pays                                                      212 non-null    object 
 1   Produit                                                   172 non-null    object 
 2   Année                                                     172 non-null    float64
 3   Disponibilité de protéines en quantité (g/personne/jour)  172 non-null    float64
 4   Importations - Quantité                                   170 non-null    float64
 5   Production                                                168 non-null    float64
 6   Disponibilité intérieure                                  170 non-null    float64
 7   Nourriture                                                170 non-null    float64
 8   Population          

Quel est le pourcentage de valeur manquante par colonne ?

In [192]:
final_df.isna().mean()

Pays                                                        0.000000
Produit                                                     0.188679
Année                                                       0.188679
Disponibilité de protéines en quantité (g/personne/jour)    0.188679
Importations - Quantité                                     0.198113
Production                                                  0.207547
Disponibilité intérieure                                    0.198113
Nourriture                                                  0.198113
Population                                                  0.188679
Produit intérieur brut en dollars                           0.000000
dtype: float64

Quel est le nombre total de lignes dupliquée ?

In [193]:
final_df.duplicated().sum()

0

Quel est le nombre de valeurs différentes par colonne ?

In [194]:
final_df.nunique()

Pays                                                        212
Produit                                                       1
Année                                                         1
Disponibilité de protéines en quantité (g/personne/jour)    164
Importations - Quantité                                      84
Production                                                  125
Disponibilité intérieure                                    135
Nourriture                                                  130
Population                                                  172
Produit intérieur brut en dollars                           212
dtype: int64

# 8. Nettoyage des données manquantes

Pour les données manquantes, nous allons supprimer les lignes où les données sont manquantes :

In [195]:
columns = final_df.columns
unfill_df = pd.DataFrame(columns= columns)

for x in columns:
    unfill = final_df[final_df[x].isna()]
    unfill_df = pd.concat([unfill_df,unfill])

unfill_df = unfill_df.drop_duplicates()

display(unfill_df.shape)

for x in columns:
    final_df = final_df[final_df[x].notna()]

display(final_df.shape)

final_df['Produit intérieur brut en dollars'] = final_df['Produit intérieur brut en dollars']*pow(10,6)/final_df.Population
final_df = final_df.rename(columns={'Produit intérieur brut en dollars' : 'Produit intérieur brut en dollars par habitant'})
final_df['Importations - Quantité'] = final_df['Importations - Quantité']*pow(10,9)/final_df.Population
final_df = final_df.rename(columns={'Importations - Quantité' : 'Importations - gramme par habitant'})
final_df['Production'] = final_df['Production']*pow(10,9)/final_df.Population
final_df = final_df.rename(columns={'Production' : 'Production gramme par habitant'})
final_df['Disponibilité de protéines en quantité (g/personne/jour)'] = final_df['Disponibilité de protéines en quantité (g/personne/jour)']*360
final_df = final_df.rename(columns={'Disponibilité de protéines en quantité (g/personne/jour)' : 'Disponibilité de protéines en quantité (g/personne)'})
final_df['Disponibilité intérieure'] = final_df['Disponibilité intérieure']*pow(10,9)/final_df.Population
final_df = final_df.rename(columns={'Disponibilité intérieure' : 'Disponibilité intérieure gramme par habitant'})
final_df['Nourriture'] = final_df['Nourriture']*pow(10,9)/final_df.Population
final_df = final_df.rename(columns={'Nourriture' : 'Nourriture gramme par habitant'})


(44, 10)

(168, 10)

# 9. Dernier nettoyage et sauvegarde de la table finale en csv

Affichage de la table finale

In [196]:
final_df.head()

,Pays,Produit,Année,Disponibilité de protéines en quantité (g/personne),Importations - gramme par habitant,Production gramme par habitant,Disponibilité intérieure gramme par habitant,Nourriture gramme par habitant,Population,Produit intérieur brut en dollars par habitant
0,Afghanistan,Viande de Volailles,2017.0,194.4,798.983627,771.432467,1570.416094,1515.313775,36296113.0,520.616409
1,Afrique du Sud,Viande de Volailles,2017.0,5079.6,9016.000700,29240.609274,37151.535958,35695.644795,57009756.0,6121.876572
2,Albanie,Viande de Volailles,2017.0,2253.6,13175.372178,4507.364166,16295.855063,16295.855063,2884169.0,4514.204908
3,Algérie,Viande de Volailles,2017.0,709.2,48.321797,6644.247125,6692.568922,6378.477240,41389189.0,4109.696001
4,Allemagne,Viande de Volailles,2017.0,2865.6,10186.501412,18316.345769,21038.391871,19465.654124,82658409.0,44651.829102


Suppression de la colonne Année : 

In [197]:
final_df = final_df.drop(columns = ['Année','Produit','Population'])

Affichage du nouveau tableau

In [198]:
final_df.head()

,Pays,Disponibilité de protéines en quantité (g/personne),Importations - gramme par habitant,Production gramme par habitant,Disponibilité intérieure gramme par habitant,Nourriture gramme par habitant,Produit intérieur brut en dollars par habitant
0,Afghanistan,194.4,798.983627,771.432467,1570.416094,1515.313775,520.616409
1,Afrique du Sud,5079.6,9016.000700,29240.609274,37151.535958,35695.644795,6121.876572
2,Albanie,2253.6,13175.372178,4507.364166,16295.855063,16295.855063,4514.204908
3,Algérie,709.2,48.321797,6644.247125,6692.568922,6378.477240,4109.696001
4,Allemagne,2865.6,10186.501412,18316.345769,21038.391871,19465.654124,44651.829102


Sauvegarde du fichier

In [199]:
final_df.to_csv(data_set_path + 'final-file.csv',index = False, encoding= 'utf-16-le')